In [1]:
import os

from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import multiprocessing as mp
import shutil

from imblearn.keras import BalancedBatchGenerator
from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
img_size = 96
epochs = 75
batch_size = 32
testsplit = .25
targetx = 96
targety = 96
learning_rate = 0.0001
classes = 7
seed = 23
print(seed)


23


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter

model_name = 'INV3Balancedmodel.keras'

# Define two different ImageDataGenerators with their respective directories
datagen1 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

datagen2 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

train_dir1 = 'KDEFMF/MaleBalancedTraining'
train_dir2 = 'KDEFMF/FemaleBalancedTraining'
val_dir1 = 'KDEFMF/MaleBalancedValidation'
val_dir2 = 'KDEFMF/FemaleBalancedValidation'

# Create two separate generators using different directories
train_generator1 = datagen1.flow_from_directory(
    train_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)
validation_generator1 = datagen1.flow_from_directory(
    val_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
)
validation_generator2 = datagen2.flow_from_directory(
    val_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
)

train_generator2 = datagen2.flow_from_directory(
    train_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

total_train_samples = len(train_generator1.filenames) + len(train_generator2.filenames)
total_validation_samples = len(validation_generator1.filenames) + len(validation_generator2.filenames)

# Combine labels from both generators
all_labels = list(train_generator1.classes) + list(train_generator2.classes)

# Get class indices (assuming they are the same for both generators)
class_indices = train_generator1.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# Count the number of occurrences of each class
class_counts = Counter(all_labels)

# Print the number of images per class
for class_index, count in class_counts.items():
    class_name = index_to_class[class_index]
    print(f"Class '{class_name}' has {count} images.")
    
def combined_generator(gen1, gen2, num_batches):
    while True:
        try:
            batch1_x, batch1_y = next(gen1)
            batch2_x, batch2_y = next(gen2)

            combined_x = np.concatenate((batch1_x, batch2_x), axis=0)
            combined_y = np.concatenate((batch1_y, batch2_y), axis=0)

            yield combined_x, combined_y
        except StopIteration as e:
            print("Generator exhausted or encountered an error:", e)
            break


Found 1120 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 1120 images belonging to 7 classes.
Class 'angry' has 320 images.
Class 'disgust' has 320 images.
Class 'fear' has 320 images.
Class 'happy' has 320 images.
Class 'neutral' has 320 images.
Class 'sad' has 320 images.
Class 'surprise' has 320 images.


In [4]:
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten() (x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(512, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(256, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
predictions = Dense(classes, activation='softmax', kernel_initializer='random_uniform', bias_initializer='zeros')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(learning_rate=learning_rate)
loss = "categorical_crossentropy"

for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])


In [5]:
# Define the number of batches you want from each generator
num_batches_per_gen = 100  # Adjust as needed

# Use the combined generator for training
combined_train_generator = combined_generator(train_generator1, train_generator2, num_batches_per_gen)

# Use the combined generator for validation
combined_validation_generator = combined_generator(validation_generator1, validation_generator2, num_batches_per_gen)


steps_per_epoch = total_train_samples // (2 * 32)
validation_steps = total_validation_samples // (2 * 32)

# Ensure correct optimizer and loss function
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Ensure ModelCheckpoint is correctly configured used for callback to save the best value based on val_accuracy
checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model with callbacks
history = model.fit(
    combined_train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=combined_validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    callbacks=[checkpoint]
)

Epoch 1/75
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1479 - loss: 8.0257
Epoch 1: val_accuracy improved from -inf to 0.14062, saving model to INV3Balancedmodel.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.1480 - loss: 8.0051 - val_accuracy: 0.1406 - val_loss: 66.2118
Epoch 2/75
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1429 - loss: 6.2781
Epoch 2: val_accuracy improved from 0.14062 to 0.14718, saving model to INV3Balancedmodel.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.1429 - loss: 6.2750 - val_accuracy: 0.1472 - val_loss: 106.9956
Epoch 3/75
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1579 - loss: 5.8738
Epoch 3: val_accuracy improved from 0.14718 to 0.14919, saving model to INV3Balancedmodel.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.1577 - loss: 5.8718 - val_accuracy: 0.1492 - val_loss: 30.1146
Epoch 4/75
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1627 - loss: 5.5872
Epoch 4: val_accuracy did not imp

In [6]:
# Collect predictions and true labels
y_true = []
y_pred = []

for batch_x, batch_y in combined_validation_generator:
    # Make predictions
    predictions = model.predict(batch_x)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(batch_y, axis=1)
    
    # Append to lists
    y_true.extend(true_classes)
    y_pred.extend(predicted_classes)
    
    # Stop if you have processed the entire validation set
    if len(y_true) >= total_validation_samples:
        break

# Generate classification report
report = classification_report(y_true, y_pred, target_names=validation_generator1.class_indices.keys())
print(report)

2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 177ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step
              precision    recall  f1-score   support

       angry       0.59      0.34      0.43        80
     disgust       0.93      0.64      0.76        80
        fear       0.37      0.33      0.34        80
       happy       0.48      0.74      0.58        80
     neutral       0.44      0.56      0.49        80
         sad       0.36      0.24      0.29        80
    surprise       0.70      0.96      0.81        80

    accuracy                           0.54       560
   macro avg       0.55      0.54      0.53       560
weighted avg       0.55      0.54      0.53       560



In [7]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Assuming you have the model and validation generators defined
# validation_generator1 and validation_generator2
# Also assuming the target_names are the same for both generators

# Function to generate and print classification report
def generate_classification_report(generator, generator_name):
    y_true = []
    y_pred = []

    for batch_x, batch_y in generator:
        # Generate predictions
        predictions = model.predict(batch_x)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = np.argmax(batch_y, axis=1)

        # Collect predictions and true labels
        y_true.extend(true_classes)
        y_pred.extend(predicted_classes)

        # Break if processed all samples
        if len(y_true) >= generator.samples:
            break

    # Generate and print the classification report
    target_names = list(generator.class_indices.keys())
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(f"Classification Report for {generator_name}:\n{report}")

# Generate and print classification reports for both validation generators
generate_classification_report(validation_generator1, "Validation Generator 1")
generate_classification_report(validation_generator2, "Validation Generator 2")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
Classification Report for Validation Generator 1:
              precision    recall  f1-score   support

       angry       0.61      0.35      0.44        40
     disgust       1.00      0.57      0.73        40
        fear       0.50      0.40      0.44        40
       happy       0.38      0.57      0.46        40
     neutral       0.43      0.53      0.47        40
         sad       0.32      0.28      0.30        40
    surprise       0.68      1.00      0.81        40

    accuracy                           0.53       280
   macro avg       0.56      0.53      0.52       280
weighted avg       0.56      0.53      0.52       280

1/

In [8]:
Y_pred = model.predict(validation_generator1)
y_pred = np.argmax(Y_pred, axis=1)
y_true=validation_generator1.classes

Y_pred1 = model.predict(validation_generator2)
y_pred1 = np.argmax(Y_pred1, axis=1)
y_true1=validation_generator2.classes

cm_first = confusion_matrix(y_true, y_pred)

cm_second = confusion_matrix(y_true1, y_pred1)

# Print the confusion matrices
print("Confusion Matrix for First Dataset:")
print(cm_first)

print("Confusion Matrix for Second Dataset:")
print(cm_second)

9/9 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 467ms/step
Confusion Matrix for First Dataset:
[[ 1  3  6  6  9  5 10]
 [ 4  5  5  8 10  1  7]
 [ 3  2  3  9  9  5  9]
 [ 3  1  6  7  6  8  9]
 [ 4  4  2  9  7  4 10]
 [ 7  4  4  8  6  5  6]
 [ 5  2  3 12  9  4  5]]
Confusion Matrix for Second Dataset:
[[ 4  2  3  9 12  3  7]
 [ 8  6  3  9  4  5  5]
 [ 3  2  6  9 12  2  6]
 [ 2  2  7 11  3  4 11]
 [ 4  4  6  5  9  6  6]
 [ 1  7  8  4  9  2  9]
 [ 2  5  5  8  9  5  6]]


In [9]:
from keras import metrics

accuracy_male = metrics.Accuracy()
accuracy_female = metrics.Accuracy()
accuracy_male.update_state(y_true, y_pred)
accuracy_female.update_state(y_true1, y_pred1)

# Calculate the accuracy results
accuracy_result_male = accuracy_male.result().numpy()
accuracy_result_female = accuracy_female.result().numpy()

# Print the individual accuracies
print("Male Accuracy: %f" % accuracy_result_male)
print("Female Accuracy: %f" % accuracy_result_female)

# Calculate the accuracy difference
accuracy_difference = accuracy_result_male - accuracy_result_female
DP = accuracy_result_female/accuracy_result_male
# Print the accuracy difference
print("Accuracy Difference: %f" % accuracy_difference)
print("DP: %f" % DP)

Male Accuracy: 0.117857
Female Accuracy: 0.157143
Accuracy Difference: -0.039286
DP: 1.333333
